# Análise de modelo Watson Assistant

## Sumário

* [1. Bibliotecas para importação](#biblio)
* [2. Preparação do data frame](#prep)
  * [Limpeza dos logs](#limp)
* [3. Extração de amostra](#amos)
  * [Arquivo para curadoria](#arq_cura)
* [4. Data frame de curadoria](#df_cura)
* [5. Acurácia](#acur)
* [6. Matriz de confusão](#matrix)
* [7. Relatório de classificação (precisão, recall, F1-score)](#relat)
  * [Tabela de F1-score](#f1_score)
  * [Precisão X recall](#prec_recal)
* [8. Análise de confiança](#conf)
* [9. Cálculo do limite de confiança (τ)](#conf_calc)

## <a id="biblio"></a>1. Bibliotecas para importação
Instala, importa e atualiza bibliotecas e funções necessárias.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from operator import itemgetter
from datetime import datetime
from sklearn.metrics import classification_report, confusion_matrix
from amostragem import get_sample_size, sample_extractor

##  <a id="prep"></a>2. Preparação do data frame
Abre arquivo de logs extraídos.

In [ ]:
arch1 = "EXTRACAO.CSV" #indicar arquivo .csv com a extração

In [ ]:
logs = pd.read_csv(arch1)
logs

###  <a id="limp"></a>Limpeza do data frame
Seleciona logs de primeiro turno, irrelevantes, logs com inputs vazios.

In [ ]:
turn1_logs = logs[logs.turn_counter == 1] #logs de primeiro turno
irrelevant_logs = logs[logs.intent == "Irrelevant"] #logs classificados como Irrelevantes
null_input_logs = logs[logs['input'].isnull()] #logs com inputs vazios

In [ ]:
to_remove = set(list(irrelevant_logs.index) + list(null_input_logs.index))
clean_logs = logs.drop(to_remove).reset_index(drop=True)
#clean_logs

## <a id="amos"></a>3. Extração de amostra
Usa função programada para extração de amostra dos logs para curadoria humana.
Amostra baseada em 95% de índice de confiança e 5% de margem de erro.

In [ ]:
pop = len(clean_logs)
sample_size = get_sample_size(pop)
sample_table = sample_extractor(clean_logs, sample_size)
sample_table

### <a id="arq_cura"></a>Arquivo para curadoria 
Salva arquivo .csv local para curadoria e classificação manual

In [ ]:
sample_table.to_csv(str(datetime.now()) + ".csv", index=False)

## 4.  <a id="df_cura"></a>Data frame de curadoria

In [ ]:
arch2 = "cura1.csv"

In [ ]:
curated_logs = pd.read_csv(arch2, dtype={'check': str})
curated_logs.check.astype(str)
for index, row in curated_logs.iterrows():
    if row['intent_CM'] == row['intent_watson']:
        curated_logs.at[index, 'score'] = 1
        curated_logs.at[index, 'check'] = "S"
    else:
        curated_logs.at[index, 'score'] = 0
        curated_logs.at[index, 'check'] = "N"

curated_logs

## 5. <a id="acur"></a>Acurácia

In [ ]:
accuracy_data = {"S":[curated_logs[curated_logs.check == "S"]["check"].count()],
                  "N":[curated_logs[curated_logs.check == "N"]["check"].count()]}
accuracy_df = pd.DataFrame(data=accuracy_data)
accuracy_df['S'].plot.bar(figsize=(3, 5), bottom=accuracy_df['N'])
accuracy_df['N'].plot.bar(color='red').set_yticks([accuracy_data['N'][0], accuracy_data['N'][0]+accuracy_data['S'][0]])
plt.legend()
plt.show()
print("Acuracy:", round(accuracy_df["S"][0]/(accuracy_df["S"][0]+accuracy_df["N"][0]), 2))

## 6. <a id="matrix"></a>Matriz de confusão 

In [ ]:
cm_intents = list(curated_logs['intent_CM'])
wt_intents = list(curated_logs['intent_watson'])

In [ ]:
labels = curated_logs['intent_CM'].drop_duplicates().sort_values() #cria labels com as intenções testadas
output_matrix = confusion_matrix(cm_intents, wt_intents, labels=labels) #cria matriz de confusão
index_labels  = ['CM:{:}'.format(x) for x in labels] #label das intenções classificadas manualmente
column_labels = ['Watson:{:}'.format(x) for x in labels]#lavel das intenções classificadas pelo Watson

In [ ]:
plt.figure(figsize=(10, 10))
df_cm = pd.DataFrame(output_matrix, index=index_labels, columns=column_labels) #cria DF da matriz de confusão
hm = sns.heatmap(df_cm, cmap="binary", annot=True, cbar=False, linewidths=.01, linecolor='white', square=True)
hm.set_yticklabels(hm.get_yticklabels(), rotation=0)
hm.set_xticklabels(hm.get_xticklabels(), rotation=90)
hm.set_frame_on(True)
plt.title("Matriz de confusão")
plt.tight_layout()
plt.autoscale()

## <a id="relat"></a>7. Relatório de classificação (precisão, recall, F1-score)  

In [ ]:
cr = classification_report(cm_intents, wt_intents, output_dict=True) #cria relatório de indicadores de performance de modelo
cr_df = pd.DataFrame(cr).T
cr_df

### <a id="f1_score"></a>Tabela de F1-score 

In [ ]:
pd.DataFrame(cr).T["f1-score"].sort_values()

### <a id="prec_recal"></a>Precisão x recall

In [ ]:
scatter = cr_df.drop(index=["accuracy", "macro avg", "weighted avg"]).sort_values("support", ascending=False)
#scatter.plot.scatter("precision", "recall", figsize=(7,7))
sizer = {key:value*30 for key, value in scatter['support'].items()}

plt.figure(figsize=(6,6))

f_scores = np.linspace(0.2, 0.8, num=4)
lines = []
labels = []
for f_score in f_scores:
    x = np.linspace(0.01, 1)
    y = f_score * x / (2 * x - f_score)
    l, = plt.plot(x[y >= 0], y[y >= 0], color='gray', alpha=0.2)
    plt.annotate('f1={0:0.1f}'.format(f_score), xy=(0.9, y[45] + 0.02))
    
plt.axis([-0.05, 1.05, -0.05, 1.05])

for index, row in scatter.iterrows():
    if row["f1-score"] == 0:
        continue #remove intenções com suporte = 0
    plt.scatter(row['precision'], row['recall'], label=index, s=sizer[index])
plt.xlabel('precision')
plt.ylabel('recall')
plt.title("Classification Report")
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5), title="INTENÇÕES", labelspacing=4, ncol= 2, handlelength=8)
plt.show()

## <a id="conf"></a>8. Análise de confiança
Distribuição dos acertos e erros do modelo por índice de confiança

In [ ]:
no_conf_list = curated_logs[curated_logs['check']=='N']['confidence']
yes_conf_list = curated_logs[curated_logs['check']=='S']['confidence']
bins = np.linspace(0, 1, 6)

plt.figure(figsize=(15, 5))
plt.hist([no_conf_list, yes_conf_list], bins=bins, label=['incorreto', 'correto'], color=['red', 'royalblue'])
plt.legend(loc='upper left')
plt.title('Distribuição de logs por confiança')
plt.xlabel('Confiança')
plt.ylabel('Qts de logs')
plt.show()

curated_logs.boxplot(column="confidence", by="check", figsize=(3, 5), showfliers=False, grid=False)

### Baixa assertividade

In [ ]:
curated_logs[curated_logs['check'] == 'N'][curated_logs["confidence"] <= 0.85]

## <a id="conf_calc"></a>9. Cálculo do limite de confiança (τ) 

In [ ]:
threshold_results = []

correct_weight = 1
incorrect_weight = -1.5
ignored_weight = 0

print("Total:", len(curated_logs), "logs")

for i in range(0, 100):
    test_tau = i/100
    
    correct_count = len(curated_logs[(curated_logs['check'] == 'S') & (curated_logs['confidence'] >= test_tau)])
    incorrect_count = len(curated_logs[(curated_logs['check'] == 'N') & (curated_logs['confidence'] >= test_tau)])
    ignored_count = len(curated_logs[(curated_logs['confidence'] < test_tau)])
    
    score = ((correct_count*correct_weight)+(incorrect_count*incorrect_weight)+(ignored_count*ignored_weight))
    
    threshold_result = (test_tau,score)
    threshold_results.append(threshold_result)
    #print('test_tau:', threshold_result[0], "score:", threshold_result[1])

optimum_condfidence_threshold, max_score = max(threshold_results, key=itemgetter(1))
print('optimum_condfidence_threshold:', optimum_condfidence_threshold)
print('max_score:', max_score)

### Intenções conflitantes

In [ ]:
conflict_intents = {"Intents":[],
                   "Qtd":[]}
agrupado = curated_logs[(curated_logs['check'] == 'N')].sort_values(['intent_CM', 'intent_watson'])[["intent_CM", "intent_watson", "input"]]
for index, row in agrupado.iterrows():
    key = row['intent_CM'] + " / " + row['intent_watson']
    if key in conflict_intents["Intents"]:
        conflict_intents["Qtd"][conflict_intents["Intents"].index(key)] += 1
    if key not in conflict_intents["Intents"]:
        conflict_intents["Intents"].append(key)
        conflict_intents["Qtd"].append(1)
#conflict_intents
conflicted_intents = pd.DataFrame(data=conflict_intents).sort_values("Qtd", ascending=False) #Tabela com pares de intenções conflitantes
conflicted_intents[conflicted_intents["Qtd"] >= 2]

In [ ]:
to_train = curated_logs[(curated_logs["intent_CM"] != "Irrelevant") & (curated_logs["intent_watson"] != "Irrelevant") & (curated_logs['check'] == 'N')][["input", "intent_CM"]]
to_train.to_csv("treinamento.csv", index=False) #salva local planilha com exemplos para treinamento de intenções

### Irrelevantes

In [ ]:
curated_logs[curated_logs.intent_CM == "Irrelevant"]